In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, Embedding
from tensorflow.keras.layers import BatchNormalization, Activation, UpSampling2D, Conv2D, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Concatenate
import numpy as np
import matplotlib.pyplot as plt

# Parameters
img_rows = 32
img_cols = 32
channels = 3  # CIFAR-10 images are in RGB
num_classes = 10  # CIFAR-10 has 10 classes
latent_dim = 110

# Input shape
img_shape = (img_rows, img_cols, channels)

# Optimizer
optimizer = Adam(0.0002, 0.5)


In [5]:
def build_generator():
    model = Sequential()

    # The input_dim should be latent_dim, not latent_dim + num_classes
    model.add(Dense(256 * 8 * 8, activation="relu", input_dim=latent_dim))
    model.add(Reshape((8, 8, 256)))
    model.add(UpSampling2D())
    model.add(Conv2D(256, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))

    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))

    model.add(Conv2D(64, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))

    model.add(Conv2D(channels, kernel_size=3, padding="same"))
    model.add(Activation("tanh"))

    noise = Input(shape=(latent_dim,))
    label = Input(shape=(1,), dtype='int32')
    label_embedding = Flatten()(Embedding(num_classes, latent_dim)(label))

    # Element-wise multiplication, resulting in shape (None, latent_dim)
    model_input = multiply([noise, label_embedding])
    img = model(model_input)

    return Model([noise, label], img)
# Instantiate generator
generator = build_generator()


In [6]:
def build_discriminator():
    model = Sequential()

    # The input_shape should have channels + 1 to accommodate the label embedding
    model.add(Conv2D(64, kernel_size=3, strides=2, input_shape=(img_shape[0], img_shape[1], channels + 1), padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))

    model.add(Conv2D(256, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    img = Input(shape=img_shape)
    label = Input(shape=(1,), dtype='int32')

    # Create label embedding with the correct output shape
    label_embedding = Flatten()(Embedding(num_classes, img_shape[0] * img_shape[1])(label))

    # Reshape the label embedding to have a single channel
    label_embedding = Reshape((img_shape[0], img_shape[1], 1))(label_embedding)

    # Concatenate image and label embedding along the channels axis
    model_input = Concatenate(axis=-1)([img, label_embedding])

    validity = model(model_input)

    return Model([img, label], validity)

# Instantiate discriminator
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [7]:
# Generate noise and label as input
noise = Input(shape=(latent_dim,))
label = Input(shape=(1,))
img = generator([noise, label])

# Discriminator is not trainable during generator training
discriminator.trainable = False
valid = discriminator([img, label])

# Build the combined model (stacked generator and discriminator)
combined = Model([noise, label], valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)


In [8]:
# Load CIFAR-10 dataset
(X_train, y_train), (_, _) = tf.keras.datasets.cifar10.load_data()
X_train = (X_train.astype(np.float32) - 127.5) / 127.5  # Normalize to [-1, 1]
y_train = y_train.reshape(-1, 1)


170498071/170498071 [==============================] - 2s 0us/step


In [12]:
# Training parameters
batch_size = 64
epochs = 1000
sample_interval = 1000

# Training the GAN
for epoch in range(epochs):
    # Train Discriminator
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    imgs = X_train[idx]
    labels = y_train[idx]

    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    gen_labels = np.random.randint(0, num_classes, batch_size).reshape(-1, 1)
    gen_imgs = generator.predict([noise, gen_labels])

    real = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    d_loss_real = discriminator.train_on_batch([imgs, labels], real)
    d_loss_fake = discriminator.train_on_batch([gen_imgs, gen_labels], fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train Generator
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    valid_labels = np.random.randint(0, num_classes, batch_size).reshape(-1, 1)
    valid_y = np.ones((batch_size, 1))
    g_loss = combined.train_on_batch([noise, valid_labels], valid_y)

    print(f"{epoch} [D loss: {d_loss[0]} | D accuracy: {100 * d_loss[1]}] [G loss: {g_loss}]")

    if epoch % sample_interval == 0:
        sample_images(epoch, generator)


2/2 [==============================] - 0s 38ms/step


KeyError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1154, in train_step
        self.optimizer.minimize(loss, self.trainable_variables, tape=tape)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/optimizer.py", line 544, in minimize
        self.apply_gradients(grads_and_vars)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/optimizer.py", line 1223, in apply_gradients
        return super().apply_grad

In [9]:
# Function to sample and save generated images
def sample_images(epoch, generator, examples=10):
    noise = np.random.normal(0, 1, (examples, latent_dim))
    sample_labels = np.array([num for num in range(examples)]).reshape(-1, 1)
    generated_images = generator.predict([noise, sample_labels])

    fig, axs = plt.subplots(1, examples, figsize=(examples * 2, 2))
    for i in range(examples):
        axs[i].imshow((generated_images[i] * 0.5 + 0.5))  # Rescale images to [0, 1]
        axs[i].axis('off')
    plt.show()
